# Transformer






In [4]:
import torch
eigenmodel_path = "/root/eigenestimation/outputs/eigenmodels/tinystories-8M.pt"
eigenmodel = torch.load(eigenmodel_path)['model']
from eigenestimation.evaluation.top_logits import compute_jacobian
from importlib import reload
# reload compute jacobiab
import sys
sys.path.append('/root/eigenestimation')
reload(eigenestimation.evaluation.top_logits)

NameError: name 'eigenestimation' is not defined

In [5]:
import sys
sys.path.append('/root/eigenestimation')  # Ensure the module path is in sys.path

import eigenestimation.evaluation.top_logits  # Import the full module
from eigenestimation.evaluation.top_logits import compute_jacobian  # Import the function

from importlib import reload
reload(eigenestimation.evaluation.top_logits)  # Reload the entire module

# If you need to ensure the function is updated, re-import it
from eigenestimation.evaluation.top_logits import compute_jacobian


In [6]:
X_data_path = "/root/eigenestimation/outputs/top_tokens/tinystories-8M-X_data.pt"
attributions_path = "/root/eigenestimation/outputs/top_tokens/tinystories-8M-circuit_attributions.pt"
eigenmodel_path = "/root/eigenestimation/outputs/eigenmodels/tinystories-8M.pt"
X_data = torch.load(X_data_path)
attributions = torch.load(attributions_path)
eigenmodel = torch.load(eigenmodel_path)['model']
tokenizer = eigenmodel.model.tokenizer

In [7]:
circuit_idxs = range(len(attributions))
circuit_idxs = range(20)
for circuit_idx in circuit_idxs:
    print('\n\n----- circuit_idx:', circuit_idx, 'activation:', attributions[circuit_idx]['activation'], '-----')
    for example in attributions[circuit_idx]['top_examples']:

            
        
        tokens = (X_data[example['sample_id']][:example['token_id']+1])
        text = tokenizer.decode(tokens.long())
        
        jac = compute_jacobian(eigenmodel, tokens.to('cuda'), circuit_idx, device='cuda')
        top_token_idxs = abs(jac.argsort(descending=True)[:5])
        top_logits = [eigenmodel.model.tokenizer.decode(token_idx) for token_idx in top_token_idxs]
        bottom_token_idxs = jac.argsort(descending=False)[:5]
        bottom_logits = [eigenmodel.model.tokenizer.decode(token_idx) for token_idx in bottom_token_idxs]
        
        print(text.replace('\n', '\\newline'), '-->', example['value'], 'top_logits', top_logits)




----- circuit_idx: 0 activation: 0.115 -----
 and was shocked.\newline\newline"Lily, --> 13.854 top_logits [' recess', ' October', ' midday', ' lunch', 'ccoli']
 Tim worked hard to build the tall tower. When it --> 12.406 top_logits [' build', ' happen', ' will', ' make', ' Their']
 long they worked hard to get ready.\newline\newlineWhen they finally --> 12.111 top_logits [' aboard', '.', '."', ' saying', ' happening']
 saw a little girl crying. He went to her and asked her what --> 11.586 top_logits ['women', ' harmony', ' theirs', 'der', 'isms']
 the big doll. She said, "This is --> 10.115 top_logits [' fabric', 'Short', ' fabrics', ' cylinder', 'lace']


----- circuit_idx: 1 activation: 0.221 -----
 a time, there was a graceful cat named --> 16.941 top_logits [' foe', ' starve', ' died', 'keys', ' fate']
 bike. She zoomed fast and slow, --> 14.967 top_logits ['?".', ' will', ' command', ' Please', ' foe']
 but a snail was faster --> 14.447 top_logits ['place', ' my', ' Goodbye', '